# **Scraping SatuSehat News Content**

Web scraping di sini menggunakan `BeautifulSoup` karena library ini memudahkan parsing dan navigasi HTML, serta ekstraksi data seperti teks dan judul artikel. `BeautifulSoup` digunakan bersama `requests` untuk mengambil konten web.

## Set Up

In [1]:
# Install libraries
!pip install requests beautifulsoup4 pandas

## Web Scraping

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fungsi untuk mengambil judul dan teks dari artikel
def scrape_article(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Jika ada kesalahan HTTP, raise exception
    except requests.exceptions.RequestException as e:
        print(f"Error accessing {url}: {e}")
        return None, None  # Jika ada kesalahan, kembalikan None

    soup = BeautifulSoup(response.content, 'html.parser')

    # Mengambil judul artikel
    title = soup.find('title').get_text().strip() if soup.find('title') else None

    # Mengambil teks artikel
    paragraphs = soup.find_all('p')
    article_text = ' '.join([para.get_text().strip() for para in paragraphs])

    return title, article_text

# Fungsi untuk memproses data dan menyimpannya dalam format yang sesuai
def process_articles(file_path):
    # Load file CSV yang berisi kolom 'link' dan 'date'
    df_newsText = pd.read_csv(file_path)

    # Buat kolom baru untuk hasil scraping
    df_newsText['title'] = ''
    df_newsText['text'] = ''

    # List untuk menyimpan indeks baris yang gagal
    failed_indices = []

    # Lakukan scraping untuk setiap artikel
    for index, row in df_newsText.iterrows():
        url = row['link']
        print(f"Scraping article: {url}")
        title, article_text = scrape_article(url)

        if title and article_text:
            # Simpan hasil scraping ke dataframe
            df_newsText.at[index, 'title'] = title
            df_newsText.at[index, 'text'] = article_text
        else:
            # Jika gagal, tandai indeks untuk dihapus nanti
            failed_indices.append(index)

    # Hapus baris yang gagal scraping
    df_newsText.drop(failed_indices, inplace=True)

    # Simpan hasil scraping ke file CSV baru
    output_file = 'newsText_cleaned.csv'
    df_newsText[['title', 'link', 'text', 'tanggal']].to_csv(output_file, index=False)
    print(f"Scraping selesai. Hasil disimpan di {output_file}")
    return df_newsText

# Memanggil fungsi dengan file CSV yang berisi link dan tanggal
df_result_newsText = process_articles('/content/link_berita_satusehat.csv')

Scraping article: https://www.gresiksatu.com/ribuan-warga-antusias-ikuti-jalan-sehat-meriah-di-duduksampeyan-gresik/
Scraping article: https://rri.co.id/kesehatan/959404/aplikasi-satu-sehat-langkah-pemerintah-cegah-penyebaran-mpox
Error accessing https://rri.co.id/kesehatan/959404/aplikasi-satu-sehat-langkah-pemerintah-cegah-penyebaran-mpox: 403 Client Error: Forbidden for url: https://rri.co.id/kesehatan/959404/aplikasi-satu-sehat-langkah-pemerintah-cegah-penyebaran-mpox
Scraping article: https://www.kompas.id/baca/humaniora/2024/09/06/integrasi-data-kualitas-udara-dalam-aplikasi-satu-sehat
Scraping article: https://beritajateng.tv/antisipasi-penyakir-mpox-di-indonesia-pemerintah-membuat-aplikasi-satu-saehat/
Scraping article: https://radarbangkalan.jawapos.com/news/2315061525/cegah-mpox-menyebar-di-indonesia-dengan-aplikasi-satu-sehat-ini-caranya
Error accessing https://radarbangkalan.jawapos.com/news/2315061525/cegah-mpox-menyebar-di-indonesia-dengan-aplikasi-satu-sehat-ini-caranya:

In [5]:
# Menampilkan beberapa baris pertama dari hasil scraping
print(df_result_newsText.head())

                                                link     tanggal  \
0  https://www.gresiksatu.com/ribuan-warga-antusi...  2024-09-09   
2  https://www.kompas.id/baca/humaniora/2024/09/0...  2024-09-07   
3  https://beritajateng.tv/antisipasi-penyakir-mp...  2024-09-07   
5  https://regional.kompas.com/read/2024/09/06/11...  2024-09-06   
6  https://inet.detik.com/science/d-7526257/cegah...  2024-09-05   

                                               title  \
0  Ribuan Warga Antusias Ikuti Jalan Sehat Meriah...   
2  Integrasi Data Kualitas Udara dalam Aplikasi S...   
3  Aplikasi Satu Sehat untuk Mencegah Penyakit Mp...   
5  Cegah Virus Cacar Monyet, Jemaah Umrah Wajib I...   
6  Cegah Mpox Menyebar, Warga RI Wajib Pakai Apli...   

                                                text  
0  GresikSatu | Ribuan warga Desa Duduksampeyan m...  
2  Integrasi data kualitas udara dilakukan lewat ...  
3  SEMARANG, beritajateng.tv – Untuk mengatasi pe...  
5  Cegah Virus Cacar Monyet, Jemaa